In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [ ]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command: 
    # the command is used to change/update the state within the agent process
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(
        update=
        {
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)] 
        # for best practice, we can set a tool message to define what tool we have used, and what tool has been called in debugging
        }
    )

In [7]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


agent = create_agent(
    #"gpt-5-nano",
    model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="My favourite colour is green")]
    },

    config=config
)

In [ ]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='5ffbf4a6-8b78-4fcc-b76c-034ea17f9a93'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'4cbb7bae-3ff4-4b92-bbab-cbbf8ea3c6cd': 'CpoDAXLI2nzdVYlgnxPIlTf5Jo30jxi66CImXVKmJVzspWQdQ5lXjvphQ21yB4yYi6Zo8VU+nFXJ+RoThq4c0Tooh1QVt6NyNzKVSGd2wlrqm0/Tyg7omEGrNLgCv7detZcVCXviG8J4i19VWojSywFK+mjqei5LvW+BgUXmRU/h8tyuWNLngAnNZDKM6VgStVZZ3PNHvqTnundLvLzNAQiDFGDbLGLjUIJgHWLsH1Ju4fGFY4OlfOofuTuc0VNTh55jqbHR+MLy8RLyQv6067biFGl9vlaG+KHWT96KDnZ22klb0JO7q0oZPp5d9lNXCDX2f0H2m9nCnb/L7Jt3BE4FKvXaIpsMcoU13C6hGTv5uH1rujtUV00TCEaoGQViHuf+pxW+MDBS4MXSv+CQJf1aSh5fEHJNlqS9k19lDw34yXGmWsPWFcCbOeFplIPdhYAKCBY1K5NWCgiyfh6nZh4AwxjvLK3XEI7VQ794EsSxeJbhgteC4jal0PySr3pbv5tGGXu1xZ6eHfp1HKKs/gfB050C1B4q4BGbbuw='}}, response_metad

In [10]:
config1 = {"configurable": {"thread_id": "10"}}
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    config=config1
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='9a20d87d-8c96-4062-99a0-638dc37c6018'),
              AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b984f-e89d-7b82-9b6c-1b1949ca8025-0', usage_metadata={'input_tokens': 65, 'output_tokens': 16, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}})]}


## Read state

In [ ]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        # the state is being as a dictionary
        return runtime.state["favourite_colour"]
        # will return the favourite colour if it exists that is being in the state
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    #"gpt-5-nano",
    model, 
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [14]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='a2fbd7d8-d759-452f-a2a0-2aa3efc09c58'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'22454dbf-43f2-4845-9f75-fe1b58c75345': 'CokCAXLI2nyD6mJOZLmltkw3XTUi1uMzKVIh0Qte01i/RFWHLuK5dbn2ZHGoTZ4/Exa13E8uNNLAzgAxgK/aQyxCMrbsrog8K814Atf7uO/YFtBoZAWw5hNpqRhznAwGvgUsrZmoBB4GdB4bEH3RAIceEhYK+JmJj/tg4s0Lk86DbCqxGpDDMJKtSR+YLjVopXAo3hJ2eBfNVwgArWLomRIDHktAi6nl0D8BbwwsWwgCHmNCOZ7h2wCEkMRFo/c3j60BhAoKugQmuAZ18upvU6uEh8CXQP4KUAErvi/DdzEUGzFDPmXyYtP8T3Ms0OaK6IMaP7zDp+Qofr2Ujm9hbgP/Y2w8XHP3pWVCGA=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9858-0905-7be3-a7e1-b07ca2995b8a-0', tool_calls=[{'name

In [15]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='a2fbd7d8-d759-452f-a2a0-2aa3efc09c58'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'22454dbf-43f2-4845-9f75-fe1b58c75345': 'CokCAXLI2nyD6mJOZLmltkw3XTUi1uMzKVIh0Qte01i/RFWHLuK5dbn2ZHGoTZ4/Exa13E8uNNLAzgAxgK/aQyxCMrbsrog8K814Atf7uO/YFtBoZAWw5hNpqRhznAwGvgUsrZmoBB4GdB4bEH3RAIceEhYK+JmJj/tg4s0Lk86DbCqxGpDDMJKtSR+YLjVopXAo3hJ2eBfNVwgArWLomRIDHktAi6nl0D8BbwwsWwgCHmNCOZ7h2wCEkMRFo/c3j60BhAoKugQmuAZ18upvU6uEh8CXQP4KUAErvi/DdzEUGzFDPmXyYtP8T3Ms0OaK6IMaP7zDp+Qofr2Ujm9hbgP/Y2w8XHP3pWVCGA=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9858-0905-7be3-a7e1-b07ca2995b8a-0', tool_calls=[{'name